# Part 1 - Basic Walkthough
This notebook contains blocks to guide you through how to run your first spoof audio detection model, using the AASIST model and custom trained weights.

In [ ]:
# Setup

# Import the git repo and install required libraries
!git clone --branch Showcase https://github.com/Hapemo/Deepfake-Audio-Detection/
%cd Deepfake-Audio-Detection/
%pip install -r requirements.txt

# Prepare data folder and download speech samples
!mkdir data
%cd data
!gdown 1WevCjrJJ7pv9XzCwjbyJ1iNr2uoOYnZI
!gdown 1TE88aXpA5YvTP5KwjM_1SYs3_HinXjDj
!unzip showcase_samples.zip
!unzip SileroVAD_samples.zip
%cd ..


## Loading in libraries

In [ ]:
import sys
sys.path.append("pyfiles")
from sg_dataloader import custom_predict
from main import get_model
import torch
import json
import os

## Preparing pretrained model
You can indicate which pretrained model you want to use, uncomment the line below to use that config file.

In [ ]:
config_path = "config/Apple1.1.conf"
# config_path = "config/Apple1.2.conf"
# config_path = "config/AASIST.conf"

In [ ]:

# # load experiment configurations
with open(config_path, "r") as f_json:
    config = json.loads(f_json.read())
model_config = config["model_config"]

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: {}".format(device))
if device == "cpu":
    raise ValueError("GPU not detected!")

# define model architecture
model = get_model(model_config, device)

# evaluates pretrained model and exit script
model.load_state_dict(
    torch.load(config["model_path"], map_location=device))
print("Model loaded : {}".format(config["model_path"]))

## Speech inference
You can load in the audio to listen and use the model to do spoof prediction. There are some sample audios you can use, in the data folder. 

Input the relative directory of the audio file in audioPath variable to select your speech.

In [ ]:
from IPython.display import Audio
import soundfile as sf

In [ ]:
audioPath = "data/showcase_samples/bonafide/SPEAKER0021/00210023.wav"

# Load the audio file
audioData, sampleRate = sf.read(audioPath)

# Play the loaded audio
Audio(audioData, rate=sampleRate)

In [ ]:

result = "Bonafide" if custom_predict(audioData, model, device, sampleRate = sampleRate, audioLength = 4) else "Spoof"
print(result)